<a href="https://colab.research.google.com/github/FariusGitHub/DataScience/blob/master/SOUP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import bs4
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

soup = bs4.BeautifulSoup(requests.get(\
'https://www.monster.ca/jobs/search/?q=data-scientist&\
where=toronto&intcid=skr_navigation_nhpso_searchMain&page=10'\
                                     ).text, 'lxml')

company=[];location=[];title=[];jobage=[]; jobpage=[]
for i,j in enumerate(tqdm(soup.select('.company'))):
            company  += [j.text.strip('\n')]
            location += [soup.select('.location')[i+1].text.strip('\n\r')]
            title    += [soup.select('.title')[i+3].text.strip('\r\n')]   
            jobage   += ['30 days or more' if soup.find_all(class_=\
                        'meta flex-col')[i].text[1:3]=='30' else\
                         'posted today'    if soup.find_all(class_=\
                        'meta flex-col')[i].text[1:3].isalpha() else\
                                              soup.find_all(class_=\
                        'meta flex-col')[i].text[1:3] + ' days ago']
            jobpage  +=  [soup.select('.title')[i+3].a.get('href')] 

def make_clickable(val):
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

monster = pd.DataFrame(np.asarray(list(zip(company, \
                                           location, title, jobage, jobpage))),\
          columns =['Company', 'Location', 'Job Title',\
                    'When Posted', 'Website'])\
          .drop_duplicates().reset_index(drop=True)

desc=[]
for i in tqdm(monster['Website']):
    desc += [bs4.BeautifulSoup(requests.get(i)\
                               .text, 'lxml').select('.row')[3].text]
    
monster['description']=desc
monster.style.format({'Website': make_clickable})

# monster.to_csv('monster.csv', index=False)
# os.startfile('monster.csv')

100%|██████████| 42/42 [00:25<00:00,  1.64it/s]


,Company,Location,Job Title,When Posted,Website,description
0,Prodigy Bank,"Toronto, ON",Data Scientist,3 days ago,https://job-openings.monster.ca/data-scientist-toronto-on-ca-prodigy-bank/221751235,"DescriptionThis is an exciting opportunity to join our client's Innovation and Strategy team at the ground level! You will be part of their innovation hub located in downtown Toronto where your desire for impact will only be matched by your innate ability to collaborate with other like minded individuals to come up with creative solutions to retail data science problems.In this role, you’ll have the chance to roll up your sleeves and apply data science methods and analytics to real-world business situations. The Data Scientist will play a key role in enhancing advanced analytics and machine learning capabilities within the organization. You will be responsible for the building, training, scoring and monitoring of machine learning models. You will work closely with various stakeholders throughout our business to understand business needs and develop solutions through machine learning or other advanced analytics/predictive modeling techniques.Job RequirementsMS/PHD or equivalent in Computer Science, Engineering, Statistics, Mathematics, or related technical fieldExperience in developing machine-learning algorithms, statistical and mathematical optimization models, and simulation and visualization toolsStrong understanding of regression modeling, time series analysis, cluster analysis, machine-learning concepts such as supervised and unsupervised learning, classification, random forest, neural nets, etc.Ability to identify predictive attributes of data sets and perform feature engineering to improve machine learning resultsExperience with manipulating big data sets via SQL able to process, filter and present large quantities of dataExperience in one or more programming languages (e.g. Python, R, Scala, etc.)Experience with Databricks, Tableau and AutoML tools an assetJob DetailsJob typeFull Time, EmployeePosted 3 days ago"
1,Step By Step Professional Services Inc,"Toronto, ON",Data Scientist (Fraud Detection Systems),19 days ago,https://job-openings.monster.ca/data-scientist-fraud-detection-systems-toronto-on-ca-step-by-step-professional-services-inc/221248864,"DescriptionNew Data Scientist (Fraud Detection Systems) contract requirement: We require a resource to come in for the initial planning and implementation phase of the implementation of the fraud detection system.This resource will assist with the selection of the solution and consider various options including vendor selections for off the shelf, building in house, etc.Resource will identify what data is currently available and what organization needs.Looking for a resource with strong expertise in this area. Must haves: Strong and recent experience in end to end implementation and operations of a fraud detectionsResource needs to understand what it takes to put a fraud detection solution into production.Background in fraud detection and data science a MUSTIdeal background in open source fraud solutions, claims based fraud detection and commercial product experienceExperience with Python a strong assetResource needs to have technical knowledge of fraud detection systems (not just someone who has used them in the past) and also strong communication and interpersonal skillsExperience with drafting business requirements or business use cases specifically related to fraud detection servicesFamiliar with various AI toolkits (no specifics as one has not yet been selected) The required skills for the Data Scientist/Analyst are: · Experience dealing with financial transactions in fraud detection techniques (ideally in a pay per claim model) · Familiarity with fraud detection technics (rule-based, statistical/predictive models, AI) and software tools · Strong business and data analysis skills · Have solution designing experience in Fraud Detection platform setup including infrastructu